In [1]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
dataset_path = './cut_chars'


if not os.path.exists(dataset_path):
    print(f"Wrong Path: {dataset_path}")
else:
    print(f"Correct Path: {dataset_path}")

data = []
labels = []

# preprocessing
for root, dirs, files in os.walk(dataset_path):
    folder_name = os.path.basename(root)
    
    if len(files) == 0:
        continue
    
    label = folder_name

    for filename in files:
        filepath = os.path.join(root, filename) 
        
        image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
        
        image = cv2.resize(image, (28, 28))

        image = np.where(image >= 75, 1, 0).astype(np.float32)
        
        data.append(image)
        labels.append(label)

data = np.array(data).astype('float32')
data = np.expand_dims(data, axis=1)  
labels = np.array(labels)

print(f"Data shape: {data.shape}")
print(f"Labels shape: {labels.shape}")

# nncode labels
chars = [
    '0','1','2','3','4','5','6','7','8','9',
    'A','B','C','D','E','F','G','H','J','K','L','M','N','P','Q','R','S','T','U','V','W','X','Y','Z',
    '藏','川','鄂','甘','赣','贵','桂','黑','沪','吉','冀','津','晋','京','辽','鲁','蒙','闽','宁',
    '青','琼','陕','苏','皖','湘','新','渝','豫','粤','云','浙'
]

chars_mapping = {char: idx for idx, char in enumerate(chars)}
inverse_chars_mapping = {idx: char for char, idx in chars_mapping.items()}

encoded_labels = np.array([chars_mapping[label] for label in labels])

# prepare dataset
X_train, X_test, Y_train, Y_test = train_test_split(data, encoded_labels, test_size=0.1, random_state=42, shuffle=True, stratify=labels)

# convert to tensor form
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
Y_test = torch.tensor(Y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define CNN
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 128, kernel_size=5, padding=2)  
        self.conv2 = nn.Conv2d(128, 256, kernel_size=5, padding=2)  
        self.conv3 = nn.Conv2d(256, 512, kernel_size=5, padding=2)  
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        
        self.fc1 = nn.Linear(512 * 3 * 3, 1024)  
        self.fc2 = nn.Linear(1024, 512)  
        self.fc3 = nn.Linear(512, len(chars))  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))  
        x = x.view(x.size(0), -1)  
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))  
        x = self.fc3(x)
        return x

model = CNNModel().to(device)

# define loss function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# testing
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

# save model
torch.save(model.state_dict(), 'cnn_char_recognition_model.pth')

# load model
model = CNNModel().to(device)
model.load_state_dict(torch.load('cnn_char_recognition_model.pth'))

# predict
image_path = './test_char/t9.png'
sample_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
sample_image = cv2.resize(sample_image, (28, 28))
sample_image = np.where(sample_image >= 75, 1, 0).astype(np.float32)
sample_image = np.expand_dims(sample_image, axis=0)  
sample_image = np.expand_dims(sample_image, axis=0)  
sample_image = torch.tensor(sample_image, dtype=torch.float32).to(device)

model.eval()
with torch.no_grad():
    output = model(sample_image)
    _, predicted_label = torch.max(output.data, 1)
    predicted_char = inverse_chars_mapping[predicted_label.item()]

print(f'Predicted character: {predicted_char}')


Using device: cuda
Correct Path: ./cut_chars
Data shape: (51893, 1, 28, 28)
Labels shape: (51893,)
Epoch [1/5], Loss: 0.4472
Epoch [2/5], Loss: 0.0929
Epoch [3/5], Loss: 0.0670
Epoch [4/5], Loss: 0.0539
Epoch [5/5], Loss: 0.0437
Accuracy of the model on the test images: 98.48%
Predicted character: 粤


/tmp/ipykernel_21805/2798629160.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('cnn_char_recognition_model.pth'))
